In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [9]:
train_path = "../data/complex_data.csv"
data = pd.read_csv(train_path, sep='\t', index_col=0)
data.head()

,Log Price,Log Area,Log Bedrooms,Log WC,Parking,Infrastructure,Facade,Log Floors
ID,,,,,,,,
1.698602,-0.405996,-0.574633,-1.388415,-1.123405,-0.933208,0.669215,-0.643026,-0.698957
1.424615,1.563954,0.184162,0.954846,0.927061,-0.933208,0.669215,-0.643026,0.627374
-0.844066,0.167636,0.053464,0.384194,-0.216060,-0.933208,-1.494289,1.555148,0.076897
-1.197166,-0.736022,-0.236331,-1.388415,-1.123405,-0.933208,0.669215,-0.643026,-0.698957
-0.776587,-0.146298,-0.529123,2.727455,1.335058,1.071573,0.669215,1.555148,1.054357


In [10]:
data.describe()

,Log Price,Log Area,Log Bedrooms,Log WC,Parking,Infrastructure,Facade,Log Floors
count,4764.000000,4764.000000,4764.000000,4764.000000,4764.000000,4764.000000,4764.000000,4764.000000
mean,-0.001738,-0.000004,0.000266,-0.000957,0.003535,0.014805,-0.001661,0.005422
std,0.999911,0.995137,0.998461,0.999261,1.000343,0.993868,0.999346,0.998988
min,-2.492482,-5.430634,-3.161024,-2.674523,-0.933208,-1.494289,-0.643026,-2.025287
25%,-0.618843,-0.668842,-0.351505,-1.123405,-0.933208,-1.494289,-0.643026,-0.698957
50%,-0.045966,-0.086254,0.384194,0.427712,-0.933208,0.669215,-0.643026,0.076897
75%,0.574668,0.584781,0.384194,0.427712,1.071573,0.669215,1.555148,0.627374
max,2.624948,6.825509,6.573883,5.988411,1.071573,0.669215,1.555148,6.898158


In [11]:
X = data.drop(columns=["Log Price"])
y = data["Log Price"]

# Support Vector Machine Regressor

In [12]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import expon, randint

svr = SVR()

param_distribution = [
    {'C': [0.1, 1, 10, 100], 'gamma': [0.1, 1, 10, 100], 'kernel': ['rbf']}, 
    {'C': [0.1, 1, 10, 100], 'degree': [1, 10, 100], 'coef0': [1, 2, 3, 4], 'kernel': ['poly']},
    {'C': [0.1, 1, 10, 100], 'kernel': ['linear']}, 
    {'C': [0.1, 1, 10, 100], 'gamma': [0.1, 1, 10, 100], 'coef0': [1, 2, 3, 4], 'kernel': ['sigmoid']},
]

random_search = RandomizedSearchCV(
    svr, 
    param_distributions=param_distribution, 
    n_iter=10, 
    scoring="neg_root_mean_squared_error",
    cv=5, 
    random_state=1, 
    n_jobs=-1
)

random_search.fit(X, y)

KeyboardInterrupt: 

In [13]:
cv_res = pd.DataFrame(random_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)

cv_res = cv_res[
    [
        'param_kernel', 
        'param_degree', 
        'param_coef0', 
        'param_C',
        'split0_test_score',
        'split1_test_score', 
        'split2_test_score',
        'mean_test_score'
    ]
]
score_cols = ["split0", "split1", "split2", "mean_test"]
cv_res.columns = ["kernel", "degree", "coef0", "C"] + score_cols
cv_res[score_cols] = np.sqrt(-cv_res[score_cols])

cv_res.head()

AttributeError: 'RandomizedSearchCV' object has no attribute 'cv_results_'

In [ ]:

cv_res

,kernel,degree,coef0,C,split0,split1,split2,mean_test
7,poly,1,2,10,0.904725,0.896835,0.875967,0.892591
3,sigmoid,NaN,2,1,1.928619,1.865927,2.018161,1.938576
0,sigmoid,NaN,1,1,3.264721,3.249561,3.266056,3.260122
4,sigmoid,NaN,4,0.1,3.612600,3.732645,3.699920,3.682070
6,sigmoid,NaN,2,0.1,3.780373,3.751142,3.734874,3.755510
2,poly,10,2,10,37.488373,26.674805,32.485145,32.517729
1,sigmoid,NaN,2,10,34.966699,34.728871,35.106969,34.934528
5,poly,100,3,100,NaN,NaN,NaN,NaN
8,poly,0.1,4,1,NaN,NaN,NaN,NaN
9,poly,100,3,10,NaN,NaN,NaN,NaN
